# Part 2.5. Batch Normalization

In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

## 1. Gradient Vanishing and Exploding
### 1.1. Gradient Vanishing and Exploding
**Gradient Vanishing**은 backpropagation을 하면서 기울기가 점점 옅어지면서 너무 작아지는 문제를 말한다. **Gradient Exploding**은 전파를 하면서 값이 너무 큰 값 혹은 nand값이 나오는 문제를 말한다.

### 1.2. Solution
우리는 위를 활성화 함수를 변경하거나(ex. `signoid` → `ReLU`), weight값을 잘 초기화하거나 더 작은 학습률을 쓰면 해결할 수 있다. 하지만 이는 간접적인 해결방법이다. 그것보다는 직접적인 해결책인 **batch normalization**을 쓰면 해결할 수 있다.

## 2. Internal Covariate Shift

![12-1.png](./img/12-1.png)

각 layer마다 입력과 출력을 갖고 있는데 다음 layer로 넘어가면서 convariate shift가 발생하여 분포가 변형되게 된다. layer가 더 깊어지면서 더더욱 이상하게 변형되는 문제가 발생한다.

## 3. Batch Normalization
한 batch에 대한 평균과 표준편차를 구해 정규화를 한 다음 scale과 shift를 한다.

![12-2.png](./img/12-2.png)

학습 시에 구한 평균과 표준편차를 sample mean과 variance라 하며 이를 따로 저장해놓고 학습이 끝난 후 learning mean과 variance를 구한다. $\gamma$와 $\beta$는 학습해줘야 하는 매개변수이다. 위의 과정이 pytorch에서 train mode일 때의 과정이다.

새로운 테스트 데이터에서는 위의 learning mean과 variance 그리고 최적화된 $\gamma$와 $\beta$를 사용한다. 앞의 과정이 pytorch에서 eval mode일 때의 과정이다.

## 4. MNIST Example

In [0]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [0]:
# parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 32

In [0]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

In [0]:
# nn layers
linear1 = torch.nn.Linear(784, 32, bias=True)
linear2 = torch.nn.Linear(32, 32, bias=True)
linear3 = torch.nn.Linear(32, 10, bias=True)
relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(32)
bn2 = torch.nn.BatchNorm1d(32)

In [0]:
# model
# batch normalization은 활성화 함수 전에!
bn_model = torch.nn.Sequential(linear1, bn1, relu,
                            linear2, bn2, relu,
                            linear3).to(device)

In [0]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
bn_optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)

In [0]:
train_losses = []
train_accs = []

valid_losses = []
valid_accs = []

train_total_batch = len(train_loader)
test_total_batch = len(test_loader)

In [21]:
for epoch in range(training_epochs):
    bn_model.train()  # set the model to train mode

    for X, Y in train_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        bn_optimizer.zero_grad()
        bn_prediction = bn_model(X)
        bn_loss = criterion(bn_prediction, Y)
        bn_loss.backward()
        bn_optimizer.step()

    with torch.no_grad():
        bn_model.eval()     # set the model to evaluation mode
        bn_loss, bn_acc = 0, 0
        for i, (X, Y) in enumerate(train_loader):
            X = X.view(-1, 28 * 28).to(device)
            Y = Y.to(device)

            bn_prediction = bn_model(X)
            bn_correct_prediction = torch.argmax(bn_prediction, 1) == Y
            bn_loss += criterion(bn_prediction, Y)
            bn_acc += bn_correct_prediction.float().mean()

        bn_loss, bn_acc = bn_loss / train_total_batch, bn_acc / train_total_batch

      
        print('[Epoch %d-TRAIN] Batchnorm Loss(Acc): bn_loss:%.5f(bn_acc:%.2f)' % (
            (epoch + 1), bn_loss.item(), bn_acc.item()))
        
        # Test the model using test sets
        bn_loss, bn_acc = 0, 0
        for i, (X, Y) in enumerate(test_loader):
            X = X.view(-1, 28 * 28).to(device)
            Y = Y.to(device)

            bn_prediction = bn_model(X)
            bn_correct_prediction = torch.argmax(bn_prediction, 1) == Y
            bn_loss += criterion(bn_prediction, Y)
            bn_acc += bn_correct_prediction.float().mean()

        bn_loss, bn_acc = bn_loss / test_total_batch, bn_acc / test_total_batch

        print('[Epoch %d-VALID] Batchnorm Loss(Acc): bn_loss:%.5f(bn_acc:%.2f)' % (
                (epoch + 1), bn_loss.item(), bn_acc.item()))
        print()

print('Learning finished')

[Epoch 1-TRAIN] Batchnorm Loss(Acc): bn_loss:0.09255(bn_acc:0.97)
[Epoch 1-VALID] Batchnorm Loss(Acc): bn_loss:0.11553(bn_acc:0.96)

[Epoch 2-TRAIN] Batchnorm Loss(Acc): bn_loss:0.08539(bn_acc:0.97)
[Epoch 2-VALID] Batchnorm Loss(Acc): bn_loss:0.10521(bn_acc:0.97)

[Epoch 3-TRAIN] Batchnorm Loss(Acc): bn_loss:0.07256(bn_acc:0.98)
[Epoch 3-VALID] Batchnorm Loss(Acc): bn_loss:0.09971(bn_acc:0.97)

[Epoch 4-TRAIN] Batchnorm Loss(Acc): bn_loss:0.06397(bn_acc:0.98)
[Epoch 4-VALID] Batchnorm Loss(Acc): bn_loss:0.09135(bn_acc:0.97)

[Epoch 5-TRAIN] Batchnorm Loss(Acc): bn_loss:0.06009(bn_acc:0.98)
[Epoch 5-VALID] Batchnorm Loss(Acc): bn_loss:0.09565(bn_acc:0.97)

[Epoch 6-TRAIN] Batchnorm Loss(Acc): bn_loss:0.05988(bn_acc:0.98)
[Epoch 6-VALID] Batchnorm Loss(Acc): bn_loss:0.09841(bn_acc:0.97)

[Epoch 7-TRAIN] Batchnorm Loss(Acc): bn_loss:0.05361(bn_acc:0.98)
[Epoch 7-VALID] Batchnorm Loss(Acc): bn_loss:0.09009(bn_acc:0.97)

[Epoch 8-TRAIN] Batchnorm Loss(Acc): bn_loss:0.05135(bn_acc:0.98)
[Ep